In [2]:
import $exclude.`org.slf4j:slf4j-log4j12`, $ivy.`org.slf4j:slf4j-nop:1.7.21` // for cleaner logs
//import $profile.`hadoop-2.6`
import $ivy.`org.apache.spark::spark-sql:2.1.0` // adjust spark version - spark >= 2.0
//import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`
import $ivy.`org.jupyter-scala::spark:0.4.2` // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)

import org.apache.spark._
import org.apache.spark.sql._
import jupyter.spark.session._
import Resolvers._

import $exclude.$                        , $ivy.$                            // for cleaner logs
//import $profile.`hadoop-2.6`

import $ivy.$                                   // adjust spark version - spark >= 2.0
//import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`

import $ivy.$                                // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)


import org.apache.spark._

import org.apache.spark.sql._

import jupyter.spark.session._

import Resolvers._

In [3]:
import $ivy.`org.vegas-viz::vegas:0.3.9`
import vegas._
import vegas.data.External._

import $ivy.$                           

import vegas._

import vegas.data.External._

In [4]:
interp.resolvers() = interp.resolvers() :+ Resolver.Http(
  "isarn project",
  "https://dl.bintray.com/isarn/maven/",
  MavenPattern,
  true
)
// resolvers += "Will's bintray" at "https://dl.bintray.com/willb/maven/"
interp.resolvers() = interp.resolvers() :+ Resolver.Http(
  "Will Benton",
  "https://dl.bintray.com/willb/maven/",
  MavenPattern,
  true
)

In [5]:
import $ivy.`org.isarnproject::isarn-sketches:0.0.3.rc1` // "org.isarnproject" %% "isarn-sketches" % "0.0.2"
import org.isarnproject.sketches._

Downloaded https://dl.bintray.com/isarn/maven/org/isarnproject/isarn-sketches_2.11/0.0.3.rc1/isarn-sketches_2.11-0.0.3.rc1.pom.sha1
Downloaded https://dl.bintray.com/isarn/maven/org/isarnproject/isarn-sketches_2.11/0.0.3.rc1/isarn-sketches_2.11-0.0.3.rc1.pom
Downloaded https://dl.bintray.com/isarn/maven/org/isarnproject/isarn-sketches_2.11/0.0.3.rc1/
Downloaded https://dl.bintray.com/isarn/maven/org/isarnproject/isarn-algebra-api_2.11/0.0.1/isarn-algebra-api_2.11-0.0.1.pom.sha1
Downloaded https://dl.bintray.com/isarn/maven/org/isarnproject/isarn-collections_2.11/0.0.1/isarn-collections_2.11-0.0.1.pom.sha1
Downloaded https://dl.bintray.com/isarn/maven/org/isarnproject/isarn-collections_2.11/0.0.1/isarn-collections_2.11-0.0.1.pom
Downloaded https://dl.bintray.com/isarn/maven/org/isarnproject/isarn-collections_2.11/0.0.1/
Downloaded https://dl.bintray.com/isarn/maven/org/isarnproject/isarn-algebra-api_2.11/0.0.1/isarn-algebra-api_2.11-0.0.1.pom
Downloaded https://dl.bintray.com/isarn/mave

import $ivy.$                                            // "org.isarnproject" %% "isarn-sketches" % "0.0.2"

import org.isarnproject.sketches._

In [7]:
import $ivy.`com.redhat.et::silex:0.1.2` // "com.redhat.et" %% "silex" % "0.1.2"

Downloaded https://dl.bintray.com/willb/maven/com/redhat/et/silex_2.11/0.1.2/silex_2.11-0.1.2.pom.sha1
Downloaded https://dl.bintray.com/willb/maven/com/redhat/et/silex_2.11/0.1.2/silex_2.11-0.1.2.pom
Downloaded https://dl.bintray.com/willb/maven/com/redhat/et/silex_2.11/0.1.2/
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-mllib_2.11/2.1.0/spark-mllib_2.11-2.1.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-mllib_2.11/2.1.0/spark-mllib_2.11-2.1.0.pom
Downloaded https://repo1.maven.org/maven2/org/joda/joda-convert/1.7/joda-convert-1.7.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze-natives_2.11/0.12/breeze-natives_2.11-0.12.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze-natives_2.11/0.12/breeze-natives_2.11-0.12.pom
Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze_2.11/0.12/breeze_2.11-0.12.pom
Downloaded https://repo1.maven.org/maven2/org/joda/joda-convert/1.7/joda-convert-1.7.pom

Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-graphx_2.11/2.1.0/spark-graphx_2.11-2.1.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-graphx_2.11/2.1.0/spark-graphx_2.11-2.1.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/jpmml/pmml-model/1.2.15/pmml-model-1.2.15.pom
Downloaded https://repo1.maven.org/maven2/org/jpmml/pmml-model/1.2.15/pmml-model-1.2.15.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/core/1.1.2/core-1.1.2.pom.sha1
Downloaded https://repo1.maven.org/maven2/net/sourceforge/f2j/arpack_combined_all/0.1/arpack_combined_all-0.1.pom
Downloaded https://repo1.maven.org/maven2/com/github/rwl/jtransforms/2.4.0/
Downloaded https://repo1.maven.org/maven2/org/spire-math/spire_2.11/0.7.4/
Downloaded https://repo1.maven.org/maven2/net/sf/opencsv/opencsv/2.3/
Downloaded https://repo1.maven.org/maven2/net/sourceforge/f2j/arpack_combined_all/0.1/arpack_combined_all-0.1.pom.sha1
Downloaded https://repo1.maven.o

Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/netlib-native_system-linux-i686/1.1/netlib-native_system-linux-i686-1.1.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/netlib-native_system-linux-x86_64/1.1/netlib-native_system-linux-x86_64-1.1.pom
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/netlib-native_system-osx-x86_64/1.1/netlib-native_system-osx-x86_64-1.1.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/jpmml/pmml-schema/1.2.15/pmml-schema-1.2.15.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/netlib-native_system-linux-x86_64/1.1/netlib-native_system-linux-x86_64-1.1.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/netlib-native_system-win-x86_64/1.1/netlib-native_system-win-x86_64-1.1.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/netlib-native_ref-linux-x86_64/1.1/netlib-native_ref-linux-x86_64-1.1.pom
Downloaded https:

Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/native_system/1.1/
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/native_ref-xbuilds/1.1/native_ref-xbuilds-1.1.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/native_system-xbuilds/1.1/
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/native_ref-xbuilds/1.1/
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/native_system-java/1.1/native_system-java-1.1.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/native_ref-java/1.1/native_ref-java-1.1.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/native_system-java/1.1/native_system-java-1.1.pom
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/native_ref-java/1.1/native_ref-java-1.1.pom
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/native_system-java/1.1/
Downloaded https://repo1.maven.org/maven2/com/gith

Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/netlib-native_system-win-x86_64/1.1/netlib-native_system-win-x86_64-1.1-natives.jar.sha1
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/netlib-native_system-linux-i686/1.1/netlib-native_system-linux-i686-1.1-natives.jar.sha1
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/netlib-native_ref-linux-armhf/1.1/netlib-native_ref-linux-armhf-1.1-natives.jar.sha1
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/native_system-java/1.1/native_system-java-1.1.jar
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/netlib-native_system-win-x86_64/1.1/netlib-native_system-win-x86_64-1.1-natives.jar
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/netlib-native_system-linux-i686/1.1/netlib-native_system-linux-i686-1.1-natives.jar
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/native_system-java/1.1/native_system-java-1.1.jar.

Downloaded https://repo1.maven.org/maven2/org/spire-math/spire_2.11/0.7.4/spire_2.11-0.7.4.jar.sha1
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-mllib_2.11/2.1.0/spark-mllib_2.11-2.1.0.jar
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/netlib-native_ref-win-i686/1.1/netlib-native_ref-win-i686-1.1-natives.jar.sha1
Downloaded https://repo1.maven.org/maven2/net/sourceforge/f2j/arpack_combined_all/0.1/arpack_combined_all-0.1.jar
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/netlib-native_ref-linux-i686/1.1/netlib-native_ref-linux-i686-1.1-natives.jar.sha1
Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze_2.11/0.12/breeze_2.11-0.12.jar
Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze-natives_2.11/0.12/breeze-natives_2.11-0.12.jar
Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze-natives_2.11/0.12/breeze-natives_2.11-0.12.jar.sha1
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netli

import $ivy.$                            // "com.redhat.et" %% "silex" % "0.1.2"

In [8]:
implicit class TDEnhance(td: TDigest) extends Serializable {
  def toXY: Vector[(Double, Double)] = {
    val q = (0.001 +: (0.05 to 0.95 by 0.05) :+ 0.999).toVector
    q.map(q => (td.cdfInverse(q), q))
  }
}

defined class TDEnhance

In [9]:
val spark = JupyterSparkSession.builder() // important - call this rather than SparkSession.builder()
  .jupyter() // this method must be called straightaway after builder()
  .master("spark://frclust:7077")
  .appName("notebook")
  .getOrCreate()
import spark.implicits._

Downloaded https://repo1.maven.org/maven2/org/jupyter-scala/spark-stubs-2_2.11/0.4.2/spark-stubs-2_2.11-0.4.2.pom
Downloaded https://repo1.maven.org/maven2/org/jupyter-scala/spark-stubs-2_2.11/0.4.2/spark-stubs-2_2.11-0.4.2.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/jupyter-scala/spark-stubs-2_2.11/0.4.2/
Downloaded https://repo1.maven.org/maven2/org/jupyter-scala/spark-stubs-2_2.11/0.4.2/spark-stubs-2_2.11-0.4.2.jar.sha1
Downloaded https://repo1.maven.org/maven2/org/jupyter-scala/spark-stubs-2_2.11/0.4.2/spark-stubs-2_2.11-0.4.2.jar


Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-yarn_2.11/2.1.0/spark-yarn_2.11-2.1.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-repl_2.11/2.1.0/spark-repl_2.11-2.1.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-repl_2.11/2.1.0/spark-repl_2.11-2.1.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-yarn_2.11/2.1.0/spark-yarn_2.11-2.1.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-repl_2.11/2.1.0/
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-yarn_2.11/2.1.0/
Downloaded https://repo1.maven.org/maven2/jline/jline/2.12.1/jline-2.12.1.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/spark-project/hive/hive-exec/1.2.1.spark2/hive-exec-1.2.1.spark2.pom.sha1
Downloaded https://repo1.maven.org/maven2/jline/jline/2.12.1/jline-2.12.1.pom
Downloaded https://repo1.maven.org/maven2/org/spark-project/hive/hive-exec/1.2.1.spark2/hive-exec-1.2.1.spark2.pom
Do

Downloaded https://repo1.maven.org/maven2/commons-lang/commons-lang/2.6/
Downloaded https://repo1.maven.org/maven2/com/twitter/parquet-hadoop-bundle/1.6.0/
Downloaded https://repo1.maven.org/maven2/com/google/inject/extensions/guice-servlet/3.0/
Downloaded https://repo1.maven.org/maven2/org/apache/avro/avro-parent/1.7.5/avro-parent-1.7.5.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/google/inject/extensions/extensions-parent/3.0/extensions-parent-3.0.pom
Downloaded https://repo1.maven.org/maven2/com/google/inject/extensions/extensions-parent/3.0/extensions-parent-3.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/apache/avro/avro-parent/1.7.5/avro-parent-1.7.5.pom
Downloaded https://repo1.maven.org/maven2/com/twitter/parquet/1.6.0/parquet-1.6.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/25/commons-parent-25.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/google/inject/extensions/extensions-parent/3.0/
Downloaded https://repo1

https://repo1.maven.org/maven2/org/spark-project/hive/hive-exe… (87.46 %, 10057…

Downloaded https://repo1.maven.org/maven2/org/spark-project/hive/hive-exec/1.2.1.spark2/hive-exec-1.2.1.spark2.jar
log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


spark: SparkSession = org.apache.spark.sql.SparkSession@537166a5

In [10]:
val raw = spark.sparkContext.parallelize(Vector.fill(1000000) { scala.util.Random.nextGaussian() }, 10)

raw: rdd.RDD[Double] = ParallelCollectionRDD[0] at parallelize at cmd9.sc:1

In [11]:
val td = raw.aggregate(TDigest.empty())(
    (td, x) => td + x,
    (td1, td2) => td1 ++ td2
)

td: TDigest = TDigest(
  0.5,
  0,
  122,
  Map(
    -4.950767382417476 -> 1.0,
    -4.75262540859294 -> 1.0,
    -4.496845233500876 -> 1.312495686613413,
    -4.483160266632817 -> 0.687504313386587,
    -4.365557233574887 -> 1.7683131939293573,
    -4.332441069983409 -> 2.9972985498500595,
    -4.234968403017813 -> 5.316344670852642,
...

In [12]:
Vegas("CDF")
    .withXY(td.toXY)
    .encodeX("x", Quant)
    .encodeY("y", Quant)
    .mark(Line)
    .show

<iframe id="frame-vegas-a91ec3c0-f27f-4740-a808-8c38fb117a91" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "x",
 "type" : "quantitative"
 },
 "y" : {
 "field" : "y",
 "type" : "quantitative"
 }
 },
 "description" : "CDF",
 "data" : {
 "values" : [
 {
 "x" : -3.0855075224520405,
 "y" : 0.001
 },
 {
 "x" : -1.645257615705565,
 "y" : 0.05
 },
 {
 "x" : -1.2850529027117081,
 "y" : 0.1
 },
 {
 "x" : -1.0374097961090067,
 "y" : 0.15000000000000002
 },
 {
 "x" : -0.8327381575799142,
 "y" : 0.2
 },
 {
 "x" : -0.6776960867449852,
 "y" : 0.25
 },
 {
 "x" : -0.5201922576975282,
 "y" : 0.3
 },
 {
 "x" : -0.38075268441894516,
 "y" : 0.35
 },
 {
 "x" : -0.24787252397300566,
 "y" : 0.39999999999999997
 },
 {
 "x" : -0.13521130050794525,
 "y" : 0.44999999999999996
 },
 {
 "x" : 2.116454820519087E-4,
 "y" : 0.49999999999999994
 },
 {
 "x" : 0.12733074252162094,
 "y" : 0.5499999999999999
 },
 {
 "x" : 0.2529268589283061,
 "y" : 0.6
 },
 {
 "x" : 0.3826524784737029,
 "y" : 0.65
 },
 {
 "x" : 0.524492309077731,
 "y" : 0.7000000000000001
 },
 {
 "x" : 0.6767118642136322,
 "y" : 0.7500000000000001
 },
 {
 "x" : 0.8439097284636773,
 "y" : 0.8000000000000002
 },
 {
 "x" : 1.0368259621622875,
 "y" : 0.8500000000000002
 },
 {
 "x" : 1.2827708200978245,
 "y" : 0.9000000000000002
 },
 {
 "x" : 3.09192630561967,
 "y" : 0.999
 }
 ]
 }
}
 }
 vg.embed("#vegas-a91ec3c0-f27f-4740-a808-8c38fb117a91", embedSpec, function(error, result) {});
 </script>
 <div id='vegas-a91ec3c0-f27f-4740-a808-8c38fb117a91'></div>
 </body>
</html>">

In [13]:
def loadCSV(fname: String) = spark.read
  .format("com.databricks.spark.csv")
  .option("header", "true")
  .option("mode", "DROPMALFORMED")
  .load(fname)

defined function loadCSV

In [20]:
val trainCSV = loadCSV("/data/tox21/tox21_dense_train.csv")

trainCSV: DataFrame = [_c0: string, AW: string ... 800 more fields]

In [21]:
trainCSV.columns

res20: Array[String] = Array(
  "_c0",
  "AW",
  "AWeight",
  "Arto",
  "BertzCT",
  "Chi0",
  "Chi1",
  "Chi10",
  "Chi2",
  "Chi3",
  "Chi3c",
...

In [22]:
val trainFV = trainCSV.rdd.map(_.toSeq.toVector.drop(1).map(_.asInstanceOf[String].toDouble)).cache()

trainFV: RDD[Vector[Double]] = MapPartitionsRDD[22] at map at cmd21.sc:1

In [23]:
import org.apache.spark.rdd.RDD
def tdSketchFV(fv: RDD[Vector[Double]]): Vector[TDigest] = {
    val tds = fv.aggregate(Array.empty[TDigest])(
        (tdv, xv) => {
            if (tdv.isEmpty) {
                Array.tabulate(xv.length) { j => TDigest.empty(maxDiscrete=50) + xv(j) }
            } else {
                for { j <- 0 until xv.length } { tdv(j) += xv(j) }
                tdv
            }
        },
        (tdv1, tdv2) => {
            if (tdv1.isEmpty) tdv2 else {
                for { j <- 0 until tdv1.length } { tdv1(j) ++= tdv2(j) }
                tdv1
            }
        }
    )
    tds.toVector
}

import org.apache.spark.rdd.RDD

defined function tdSketchFV

In [24]:
def synthesize(tdVec: Vector[TDigest], n: Int, partitions: Int = 2) = {
    implicit class AddSampling(td: TDigest) {
        import org.isarnproject.collections.mixmaps.nearest.Cover
        def cdfDiscreteInverse(q: Double): Double = {
            td.clusters.mCover(q * td.clusters.sum).map(n => (n.data.key, n.data.value)) match {
                case Cover(_, Some((x, _))) => x
                case Cover(Some((x, _)), None) => x
            }
        }

        def sample: Double = {
            val clust = td.clusters
            td.nclusters match {
                case 0 => 0.0
                case 1 => clust(clust.keyMin.get)
                case n if (n <= td.maxDiscrete) => cdfDiscreteInverse(scala.util.Random.nextDouble)
                case _ => td.cdfInverse(scala.util.Random.nextDouble)
            }
        }
    }
    val tdVecBC = spark.sparkContext.broadcast(tdVec)
    spark.sparkContext.parallelize(1 to n, partitions).map { r => tdVecBC.value.map(_.sample) }
}

defined function synthesize

In [25]:
val fvSketches = tdSketchFV(trainFV)


fvSketches: Vector[TDigest] = Vector(
  TDigest(
    0.5,
    50,
    70,
    Map(
      1.0 -> 9.0,
      1.333 -> 24.0,
      1.5 -> 45.0,
      1.600489455071117 -> 33.242848970251714,
      1.6757376364045966 -> 7.917995937532979,
      1.6979655638078115 -> 63.83915509221531,
...

In [26]:
val synthFV = synthesize(fvSketches, 48000, 4)

synthFV: RDD[Vector[Double]] = MapPartitionsRDD[24] at map at cmd23.sc:22

In [27]:
import com.redhat.et.silex.util.vectors.implicits._
import org.apache.spark.mllib.tree.RandomForest
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.regression.LabeledPoint
import com.redhat.et.silex.cluster.ClusteringTreeModel
import ClusteringTreeModel._

import com.redhat.et.silex.util.vectors.implicits._

import org.apache.spark.mllib.tree.RandomForest

import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

import org.apache.spark.mllib.regression.LabeledPoint

import com.redhat.et.silex.cluster.ClusteringTreeModel

import ClusteringTreeModel._

In [28]:
val trainLab = trainFV.map(_.toLabeledPoint(1.0))
val synthLab = synthFV.map(_.toLabeledPoint(0.0))
val trainFR = (trainLab ++ synthLab).cache()

trainLab: RDD[LabeledPoint] = MapPartitionsRDD[25] at map at cmd27.sc:1
synthLab: RDD[LabeledPoint] = MapPartitionsRDD[26] at map at cmd27.sc:2
trainFR: RDD[LabeledPoint] = UnionRDD[27] at $plus$plus at cmd27.sc:3

In [29]:
def frClassifier = {
val rfFR = RandomForest.trainClassifier(
    trainFR,              // training data
    2,                    // number of classes
    Map.empty[Int, Int],  // category info
    100,                  // forest size
    "auto",               // 
    "gini",               // split quality measure
    10,                   // max depth
    20,                   // max bins
    235711)               // random seed
val predictionAndLabels = trainLab.map { case LabeledPoint(label, features) =>
  val prediction = rfFR.predict(features)
  (prediction, label)
}
val metrics = new BinaryClassificationMetrics(predictionAndLabels)
val incorrect = predictionAndLabels.filter { case (p, t) => p != t }
println(s"incorrect = ${incorrect.count}")
(rfFR, metrics)
}

defined function frClassifier

In [30]:
val (rfFR, metrics) = frClassifier

incorrect = 0


rfFR: org.apache.spark.mllib.tree.model.RandomForestModel = TreeEnsembleModel classifier with 100 trees

metrics: BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@2acf5316

In [31]:
val frRules = rfFR.trees.map(_.rules(trainCSV.columns.toVector.drop(1), Map.empty[Int, Int]))

frRules: Array[Map[Double, Seq[Seq[Predicate]]]] = Array(
  Map(
    1.0 -> ArrayBuffer(
      List((MRVSA2 <= 0.0)),
      List((MRVSA2 > 0.0), (RDFM30 <= 0.0)),
      List((MRVSA2 > 0.0), (RDFM30 > 0.0), (PEOEVSA4 <= 0.0)),
      List((MRVSA2 > 0.0), (RDFM30 > 0.0), (PEOEVSA4 > 0.0), (Chiv4c <= 0.0)),
      List(
        (MRVSA2 > 0.0),
        (RDFM30 > 0.0),
        (PEOEVSA4 > 0.0),
        (Chiv4c > 0.0),
...

In [32]:
val featPerTree = for {
    rmap <- frRules
    feats = {
        val fraw = for {
            vrules <- rmap.values
            plist <- vrules
            pred <- plist
        } yield { pred.feature }
        fraw.toSet.toSeq
    }
    f <- feats
} yield { f }
val featCounts = featPerTree.foldLeft(Map.empty[String, Int])((m, f) => m + (f -> (1 + m.getOrElse(f, 0))))
val featHist = featCounts.toSeq.sortBy { case (_, n) => -n}.toVector
val featSelect = featHist.filter { case (_, n) => n > 1 }.map { case (f, _) => f }
println(featHist.mkString("\n"))

(RNCS,20)
(VSAEstate2,19)
(MRVSA7,19)
(VSAEstate3,18)
(VSAEstate0,17)
(slogPVSA8,17)
(PEOEVSA4,13)
(RDFU27,12)
(RDFM29,11)
(RDFV30,11)
(RDFM30,11)
(slogPVSA6,11)
(slogPVSA3,10)
(RDFU29,9)
(PEOEVSA10,9)
(Chi4c,8)
(PEOEVSA3,8)
(RDFE30,8)
(PEOEVSA12,7)
(Chi5ch,7)
(PEOEVSA11,7)
(slogPVSA9,7)
(Chiv4c,7)
(RDFV24,7)
(RDFU30,7)
(MRVSA8,6)
(slogPVSA11,6)
(MRVSA2,6)
(RDFV29,6)
(RDFM27,6)
(RDFE28,6)
(RDFE29,6)
(RDFV27,5)
(RDFP25,5)
(slogPVSA0,5)
(RDFP29,5)
(slogPVSA7,5)
(RDFP30,5)
(RDFM24,5)
(RDFU25,5)
(RDFP24,4)
(RDFM22,4)
(RDFV23,4)
(RDFM25,4)
(EstateVSA6,4)
(MRVSA3,4)
(Chiv5ch,4)
(RDFE27,4)
(RDFP28,4)
(RDFE25,4)
(RDFU28,3)
(EstateVSA5,3)
(RDFE26,3)
(EstateVSA2,3)
(PEOEVSA1,3)
(RDFV25,3)
(RDFV28,3)
(RDFU26,3)
(PEOEVSA8,3)
(PEOEVSA13,3)
(slogPVSA2,3)
(RDFM20,3)
(EstateVSA9,3)
(MRVSA5,2)
(RDFP27,2)
(GATSm8,2)
(RDFE20,2)
(bcutp16,2)
(RDFU24,2)
(EstateVSA1,2)
(PEOEVSA2,2)
(rygr,2)
(RDFP21,2)
(RDFM28,2)
(RDFE22,2)
(E1p,2)
(Chiv4,2)
(RDFE24,2)
(RDFU22,2)
(RDFU20,2)
(Tv,2)
(RDFP23,2)
(RDFV26,2)
(bcutm

featPerTree: Array[String] = Array(
  "RDFM30",
  "VSAEstate3",
  "Chiv4c",
  "MRVSA2",
  "PEOEVSA4",
  "VSAEstate2",
  "RDFM29",
  "RDFM30",
  "Tp",
  "Tu",
  "RNCS",
...
featCounts: Map[String, Int] = Map(
  "MRVSA8" -> 6,
  "PEOEVSA12" -> 7,
  "RDFM18" -> 1,
  "VSAEstate2" -> 19,
  "MRVSA5" -> 2,
  "Chi5ch" -> 7,
  "RDFP6" -> 1,
  "RDFM29" -> 11,
  "RDFP27" -> 2,
  "L1p" -> 1,
  "RDFP24" -> 4,
...
featHist: Vector[(String, Int)] = Vector(
  ("RNCS", 20),
  ("VSAEstate2", 19),
  ("MRVSA7", 19),
  ("VSAEstate3", 18),
  ("VSAEstate0", 17),
  ("slogPVSA8", 17),
  ("PEOEVSA4", 13),
  ("RDFU27", 12),
  ("RDFM29", 11),
  ("RDFV30", 11),
  ("RDFM30", 11),
...
featSelect: Vector[String] = Vector(
  "RNCS",
  "VSAEstate2",
  "MRVSA7",
  "VSAEstate3",
  "VSAEstate0",
  "slogPVSA8",
  "PEOEVSA4",
  "RDFU27",
  "RDFM29",
  "RDFV30",
  "RDFM30",
...

In [33]:
featSelect.length

res32: Int = 86

In [74]:
val trainLabelsCSV = loadCSV("/data/tox21/tox21_labels_train.csv")
val testLabelsCSV = loadCSV("/data/tox21/tox21_labels_test.csv")
val testCSV = loadCSV("/data/tox21/tox21_dense_test.csv")

trainLabelsCSV: DataFrame = [_c0: string, NR.AhR: string ... 11 more fields]
testLabelsCSV: DataFrame = [_c0: string, NR.AhR: string ... 11 more fields]
testCSV: DataFrame = [_c0: string, AW: string ... 800 more fields]

In [77]:
val trainCSVFR = trainCSV.select("_c0", featSelect :_*)
val testCSVFR = testCSV.select("_c0", featSelect :_*)

trainCSVFR: DataFrame = [_c0: string, RNCS: string ... 85 more fields]
testCSVFR: DataFrame = [_c0: string, RNCS: string ... 85 more fields]

In [76]:
println(trainLabelsCSV.columns.mkString(", "))

_c0, NR.AhR, NR.AR, NR.AR.LBD, NR.Aromatase, NR.ER, NR.ER.LBD, NR.PPAR.gamma, SR.ARE, SR.ATAD5, SR.HSE, SR.MMP, SR.p53


In [92]:
def csvToPairs(csv: DataFrame) =
    csv.rdd.map(_.toSeq)
    .map(v => (v.head.asInstanceOf[String], v.tail.toVector.map(_.asInstanceOf[String].toDouble)))
def lpFR(tlab: String, labCSV: DataFrame, fvCSV: DataFrame) = {
    val labCol = labCSV.select("_c0", tlab)
    val labPairs = labCol.filter(labCol(tlab) =!= "NA")
        .rdd.map(_.toSeq).map { s => (s(0).asInstanceOf[String], s(1).asInstanceOf[String].toDouble) }
    val fvPairs = csvToPairs(fvCSV)
    val lp = labPairs.join(fvPairs).map { case (_, (lab, feats)) => LabeledPoint(lab, feats.toSpark) }
    lp
}
def trainAndTest(tlab: String) =
  (lpFR(tlab, trainLabelsCSV, trainCSVFR), lpFR(tlab, testLabelsCSV, testCSVFR))

defined function lpFR
defined function trainAndTest

In [93]:
val (trainLP, testLP) = trainAndTest("`NR.AhR`")

trainLP: RDD[LabeledPoint] = MapPartitionsRDD[220] at map at cmd91.sc:6
testLP: RDD[LabeledPoint] = MapPartitionsRDD[232] at map at cmd91.sc:6

In [97]:
trainLP.filter { case LabeledPoint(lab, _) => lab != 0.0 }.count

res96: Long = 981L